In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df =pd.read_csv("/content/drive/MyDrive/processed_IIoT.csv")

df.head()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,...,mqtt.conack.flags-1471199,mqtt.conack.flags-1574358,mqtt.conack.flags-1574359,mqtt.protoname-0.0,mqtt.protoname-0,mqtt.protoname-MQTT,mqtt.topic-0.0,mqtt.topic-0,mqtt.topic-Temperature_and_Humidity,Attack_type
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,...,0,0,0,0,0,0,0,0,0,DDoS_UDP
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.504340e+08,...,0,0,0,0,1,0,0,1,0,Normal
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146818063.0,1.285670e+09,...,0,0,0,0,1,0,0,1,0,Normal
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.789139e+09,...,0,0,0,0,1,0,0,1,0,Normal
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,2.880298e+09,...,0,0,0,0,1,0,0,1,0,Normal


In [ ]:
df.shape

(1927304, 96)

In [ ]:
# Transform all features into the scale of [0,1]
numeric_features = df.dtypes[df.dtypes != 'object'].index


numeric_features

Index(['arp.opcode', 'arp.hw.size', 'icmp.checksum', 'icmp.seq_le',
       'icmp.unused', 'http.content_length', 'http.response', 'http.tls_port',
       'tcp.ack', 'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin',
       'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack',
       'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.seq', 'udp.stream',
       'udp.time_delta', 'dns.qry.name', 'dns.qry.qu', 'dns.qry.type',
       'dns.retransmission', 'dns.retransmit_request',
       'dns.retransmit_request_in', 'mqtt.conflag.cleansess', 'mqtt.conflags',
       'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as', 'mqtt.msgtype',
       'mqtt.proto_len', 'mqtt.topic_len', 'mqtt.ver', 'mbtcp.len',
       'mbtcp.trans_id', 'mbtcp.unit_id', 'http.request.method-0.0',
       'http.request.method-0', 'http.request.method-GET',
       'http.request.method-OPTIONS', 'http.request.method-POST',
       'http.request.method-PROPFIND', 'http.request.method-PUT',
       'http.request.meth

In [ ]:
scaler = QuantileTransformer()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

df[numeric_features]

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,...,mqtt.conack.flags-1471198,mqtt.conack.flags-1471199,mqtt.conack.flags-1574358,mqtt.conack.flags-1574359,mqtt.protoname-0.0,mqtt.protoname-0,mqtt.protoname-MQTT,mqtt.topic-0.0,mqtt.topic-0,mqtt.topic-Temperature_and_Humidity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.263263,0.328806,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.970366,0.420144,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.396897,0.775191,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.632132,0.787821,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1927300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.263263,0.523281,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1927301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1927302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.696196,0.599904,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
# Multiply the feature values by 255 to transform them into the scale of [0,255]
df[numeric_features] = df[numeric_features].apply(
    lambda x: (x*255))

df.describe()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,...,mqtt.conack.flags-1471198,mqtt.conack.flags-1471199,mqtt.conack.flags-1574358,mqtt.conack.flags-1574359,mqtt.protoname-0.0,mqtt.protoname-0,mqtt.protoname-MQTT,mqtt.topic-0.0,mqtt.topic-0,mqtt.topic-Temperature_and_Humidity
count,1.927304e+06,1.927304e+06,1.927304e+06,1.927304e+06,1927304.0,1.927304e+06,1.927304e+06,1927304.0,1.927304e+06,1.927304e+06,...,1.927304e+06,1.927304e+06,1.927304e+06,1.927304e+06,1.927304e+06,1.927304e+06,1.927304e+06,1.927304e+06,1.927304e+06,1.927304e+06
mean,4.763130e-03,4.763130e-03,8.902417e+00,1.030067e+01,0.0,5.805433e+00,4.313676e+00,0.0,1.232225e+02,1.234409e+02,...,2.646183e-04,2.646183e-04,1.190783e-03,1.190783e-03,7.221435e-01,1.717161e+02,1.098404e+01,7.221435e-01,1.717165e+02,1.098365e+01
std,1.102078e+00,1.102078e+00,4.637156e+01,4.962106e+01,0.0,3.783384e+01,3.288434e+01,0.0,7.881836e+01,7.930305e+01,...,2.597646e-01,2.597646e-01,5.510429e-01,5.510429e-01,1.355084e+01,1.195876e+02,5.177145e+01,1.355084e+01,1.195874e+02,5.177056e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,6.713213e+01,6.309925e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,1.236712e+02,1.279614e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.550000e+02,0.000000e+00,0.000000e+00,2.550000e+02,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,1.902928e+02,1.912444e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.550000e+02,0.000000e+00,0.000000e+00,2.550000e+02,0.000000e+00
max,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,0.0,2.550000e+02,2.550000e+02,0.0,2.550000e+02,2.550000e+02,...,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02


In [ ]:
#Generate images corresponding to each class
df0=df[df['Attack_type']=='Normal'].drop(['Attack_type'],axis=1)
df1=df[df['Attack_type']=='DDoS_UDP'].drop(['Attack_type'],axis=1)
df2=df[df['Attack_type']=='DDoS_ICMP'].drop(['Attack_type'],axis=1)
df3=df[df['Attack_type']=='SQL_injection'].drop(['Attack_type'],axis=1)
df4=df[df['Attack_type']=='DDoS_TCP'].drop(['Attack_type'],axis=1)
df5=df[df['Attack_type']=='Vulnerability_scanner'].drop(['Attack_type'],axis=1)
df6=df[df['Attack_type']=='Password'].drop(['Attack_type'],axis=1)
df7=df[df['Attack_type']=='DDoS_HTTP'].drop(['Attack_type'],axis=1)
df8=df[df['Attack_type']=='Uploading'].drop(['Attack_type'],axis=1)
df9=df[df['Attack_type']=='Backdoor'].drop(['Attack_type'],axis=1)
df10=df[df['Attack_type']=='Port_Scanning'].drop(['Attack_type'],axis=1)
df11=df[df['Attack_type']=='XSS'].drop(['Attack_type'],axis=1)
df12=df[df['Attack_type']=='Ransomware'].drop(['Attack_type'],axis=1)
df13=df[df['Attack_type']=='Fingerprinting'].drop(['Attack_type'],axis=1)
df14=df[df['Attack_type']=='MITM'].drop(['Attack_type'],axis=1)

total_images = 0

In [ ]:
# ***********Generate 95*95 color images for class 0 (Normal)************
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/Normal/"


os.makedirs(image_path, exist_ok=True)
images_cnt = 0
name = 'Normal'

for i in range(0, len(df0)):
    count=count+1
    if count<=285:
        im=df0.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt +=1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Normal images :  4828
4828


In [ ]:
# ***********Generate 95*95 color images for class 1 (DDoS_UDP)************
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/DDoS_UDP/"
os.makedirs(image_path, exist_ok=True)

name = 'DDoS_UDP'
images_cnt = 0

for i in range(0, len(df1)):
    count=count+1
    if count<=285:
        im=df1.iloc[i].values
        ims=np.append(ims,im)
    else:
        #print(im)
        #print('Total image set  = ', ims)
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt +=1
        count=0
        ims = []


total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Streaming output truncated to the last 5000 lines.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.        ]
Total image set  =  [0. 0. 0. ... 0. 0. 0.]
[  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.        239.1375993   0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        255.          0.          0.
   0.          0.          0.          0.          0.          0.
 255.          0.   

In [ ]:
# ***********Generate 95*95 color images for class 2 (DDoS_ICMP)************
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/DDoS_ICMP/"
os.makedirs(image_path, exist_ok=True)
images_cnt = 0

name = 'DDoS_ICMP'

for i in range(0, len(df2)):
    count=count+1
    if count<=285:
        im=df2.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

DDoS_ICMP images :  237
5490


In [ ]:
# ***********Generate 95*95 color images for class 3 (SQL_injection )************
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/SQL_injection/"
os.makedirs(image_path, exist_ok=True)

name = 'SQL_injection'
images_cnt = 0

for i in range(0, len(df3)):
    count=count+1
    if count<=285:
        im=df3.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt +=1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

SQL_injection images :  177
5667


In [ ]:
# ***********Generate 95*95 color images for class 4 (DDoS_TCP)************
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/DDoS_TCP/"
os.makedirs(image_path, exist_ok=True)

name = 'DDoS_TCP'
images_cnt = 0

for i in range(0, len(df4)):
    count=count+1
    if count<=285:
        im=df4.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

DDoS_TCP images :  175
5842


In [ ]:
# ***********Generate 95*95 color images for class 5 (Vulnerability_scanner)************
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/Vulnerability_scanner/"
os.makedirs(image_path, exist_ok=True)

name = 'Vulnerability_scanner'
images_cnt = 0

for i in range(0, len(df5)):
    count=count+1
    if count<=285:
        im=df5.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []


total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Vulnerability_scanner images :  174
6016


In [ ]:
# ***********Generate 95*95 color images for class 6 (Password)************
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/Password/"
os.makedirs(image_path, exist_ok=True)


name = 'Password'
images_cnt = 0

for i in range(0, len(df6)):
    count=count+1
    if count<=285:
        im=df6.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []


total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Password images :  174
6190


In [ ]:
# *Generate 95*95 color images for class 7 (DDoS_HTTP)***
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/DDoS_HTTP/"
os.makedirs(image_path, exist_ok=True)

name = 'DDoS_HTTP'
images_cnt = 0

for i in range(0, len(df7)):
    count=count+1
    if count<=285:
        im=df7.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

DDoS_HTTP images :  172
6362


In [ ]:
# *Generate 95*95 color images for class 8 (Uploading)***
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/Uploading/"
os.makedirs(image_path, exist_ok=True)

name = 'Uploading'
images_cnt = 0



for i in range(0, len(df8)):
    count=count+1
    if count<=285:
        im=df8.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Uploading images :  129
6491


In [ ]:
# ********Generate 95*95 color images for class 9 (Backdoor)***
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/Backdoor/"
os.makedirs(image_path, exist_ok=True)


name = 'Backdoor'
images_cnt = 0

for i in range(0, len(df9)):
    count=count+1
    if count<=285:
        im=df9.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []


total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Backdoor images :  84
6575


In [ ]:
# ********Generate 95*95 color images for class 10 (Port_Scanning)***
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/Port_Scanning/"
os.makedirs(image_path, exist_ok=True)

name = 'Port_Scanning'
images_cnt = 0

for i in range(0, len(df10)):
    count=count+1
    if count<=285:
        im=df10.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []


total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Port_Scanning images :  69
6644


In [ ]:
# ********Generate 95*95 color images for class 11 (XSS)***
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/XSS/"
os.makedirs(image_path, exist_ok=True)

name = 'XSS'
images_cnt = 0

for i in range(0, len(df11)):
    count=count+1
    if count<=285:
        im=df11.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

XSS images :  52
6696


In [ ]:
# ********Generate 95*95 color images for class 12 (Ransomware)***
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/Ransomware/"
os.makedirs(image_path, exist_ok=True)


name = 'Ransomware'
images_cnt = 0

for i in range(0, len(df12)):
    count=count+1
    if count<=285:
        im=df12.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Ransomware images :  33
6729


In [ ]:
# ********Generate 95*95 color images for class 13 (Fingerprinting)***
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/Fingerprinting/"
os.makedirs(image_path, exist_ok=True)

name = 'Fingerprinting'
images_cnt = 0

for i in range(0, len(df13)):
    count=count+1
    if count<=285:
        im=df13.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

Fingerprinting images :  2
6731


In [ ]:
# ********Generate 95*95 color images for class 14 (MITM)***
count=0
ims = []

image_path = "/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train/MITM/"
os.makedirs(image_path, exist_ok=True)

name = 'MITM'
images_cnt = 0

for i in range(0, len(df14)):
    count=count+1
    if count<=285:
        im=df14.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(95,95,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        images_cnt += 1
        count=0
        ims = []

total_images += images_cnt
print(f'{name} images : ' ,images_cnt)
print(total_images)

MITM images :  1
6732


In [ ]:
print(df0.shape) # Normal
print(df0.shape[0]/285)

(1380858, 95)
4845.115789473684


In [ ]:
print(df1.shape) # DDoS_UDP
print(df1.shape[0]/285)

(121567, 95)
426.55087719298245


In [ ]:
print(df2.shape) # DDoS_ICMP
print(df2.shape[0]/285)

(67939, 95)
238.38245614035088


In [ ]:
print(df3.shape) # SQL_injection
print(df3.shape[0]/285)

(50826, 95)
178.33684210526314


In [ ]:
print(df4.shape)  # DDoS_TCP
print(df4.shape[0]/285)

(50062, 95)
175.6561403508772


In [ ]:
print(df5.shape)  # Vulnerability_scanner
print(df5.shape[0]/285)

(50026, 95)
175.5298245614035


In [ ]:
print(df6.shape)  # Password
print(df6.shape[0]/285)

(49933, 95)
175.20350877192982


In [ ]:
print(df7.shape)  # DDos_HTTP
print(df7.shape[0]/285)

(49203, 95)
172.6421052631579


In [ ]:
print(df8.shape)  # Uploading
print(df8.shape[0]/285)



(36915, 95)
129.52631578947367


In [ ]:
print(df9.shape)  # Backdoor
print(df9.shape[0]/285)

(24026, 95)
84.30175438596491


In [ ]:
print(df10.shape)  # Port_Scanning
print(df10.shape[0]/285)

(19983, 95)
70.11578947368422


In [ ]:
print(df11.shape)  # XSS
print(df11.shape[0]/285)

(15066, 95)
52.863157894736844


In [ ]:
print(df12.shape)  # Ransomware
print(df12.shape[0]/285)

(9689, 95)
33.99649122807018


In [ ]:
print(df13.shape) # Fingerprinting
print(df13.shape[0]/285)

(853, 95)
2.992982456140351


In [ ]:
print(df14.shape) # MITM
print(df14.shape[0]/285)

(358, 95)
1.256140350877193
